# https://penseeartificielle.fr/methode-google-comprendre-sens-mots-word-embedding-python-gensim/

In [ ]:
phrases = [
"le chat mange ses croquettes",
"le chien aime ses croquettes",
"le chat ronronne et mange"
]

In [ ]:
#!pip install sklearn

In [ ]:
import numpy as np
import pandas as pd
 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = CountVectorizer()
vector = vectorizer.fit_transform(phrases)
 
df = pd.DataFrame(
    vector.toarray(),
    columns=vectorizer.get_feature_names(),
    index=[f"phrase_{i+1}" for i in range(len(phrases))]
)
 
print(vectorizer.get_feature_names())
print(df)

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vector = tfidf_vectorizer.fit_transform(phrases)
 
df = pd.DataFrame(
np.around(tfidf_vector.toarray(), 2),
columns=vectorizer.get_feature_names(),
index=[f"phrase_{i+1}" for i in range(len(phrases))]
)
 
print(df)

In [ ]:
from scipy.spatial.distance import cosine
cosine(
 tfidf_vectorizer.transform(['le chat ronronne et mange']).toarray(),
 tfidf_vectorizer.transform(['le chat mange ses croquettes']).toarray(),
)

In [ ]:
phrases = [
"le chat mange ses croquettes",
"le chien dévore ses croquettes",
"le chat dévore son paté",
"jean va travailler",
"le chat mange son repas",
"jacque aime quand son chien mange"
]

In [ ]:
from collections import defaultdict
contexts = defaultdict(list)
for phrase in phrases:
    words = phrase.split()
    for i, word in enumerate(words):
        candidate = [i+j for j in range (-1, 2) if j != 0]
        actual = [c for c in candidate if 0 <= c < len(words)]
        contexts[word] = contexts[word] + [words[a] for a in actual]

In [ ]:
contexts

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(phrases)
 
vectors = {}
 
for k, v in contexts.items():
    str_ = ' '.join(v)
    vectors[k] = np.around(vectorizer.transform([str_]).toarray(), 2)[0]

In [ ]:
vectors

In [ ]:
def most_similar(word):
    scores = []
    keys_ = [k for k in vectors.keys() if k != word]
    for k in keys_:
        scores.append(
          1-cosine(
              vectors[word],
              vectors[k]
          )
        )
    return pd.Series(
        scores,
        index=keys_
      ).sort_values(ascending=False)#.head(1)

In [ ]:
most_similar('paté')